In [ ]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchsummary import summary

In [ ]:
# custom module
class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = tuple(map(int,args))
    def forward(self, x):
        return x.view((x.shape[0],)+self.shape)

In [ ]:
# architecture
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # convolution layers and max pooling of encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1,16,(3,3),padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8,(3,3),padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8,8,(3,3),padding=1),
            nn.Sigmoid(),
            nn.MaxPool2d(2,padding=1),
            torch.nn.Flatten()
        )
        # convolution layers and upsampling of decoder
        self.decoder = nn.Sequential(
            Reshape(8,4,4),
            nn.Conv2d(8,8,(3,3),padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=(2,2)),
            nn.Conv2d(8,8,(3,3),padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=(2,2)),
            nn.Conv2d(8,16,(3,3)),
            nn.ReLU(),
            nn.Upsample(scale_factor=(2,2)),
            nn.Conv2d(16,1,(3,3),padding=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        # apply encoder
        features = self.encoder(x)
        # apply decoder
        return self.decoder(features)
    def __str__(self):
        return str(self.encoder)+str(self.decoder)

In [ ]:
autoencoder = Autoencoder()

In [ ]:
model_name = 'pytorch_mnist_autoencoder_model.pth'
autoencoder.load_state_dict(torch.load(model_name))

In [ ]:
summary(autoencoder, input_size=(16,1,28,28))

In [ ]:
encoder = autoencoder.encoder
encoder.eval()

In [ ]:
# test dataloader
batch_size = 512
test_loader = DataLoader(datasets.MNIST(root='data', train=False, download=True, transform=transforms.ToTensor()),batch_size=batch_size,shuffle=False)

In [ ]:
feature_vectors = []
values = []
device = 'cpu'
for batch, (x_test, y_test) in enumerate(test_loader):
    # send data to device 
    inp = x_test.to(device)
    # forward pass to the model
    y = encoder(inp)
    feature_vectors.append(y.detach().cpu().numpy())
    values += list(y_test.detach().cpu().numpy())

In [ ]:
feature_vectors = np.concatenate(feature_vectors)
print(feature_vectors.shape)

In [ ]:
colors = { 0 : (0,0,255), 1 : (0,255,255), 2 : (0,255,0), 3 : (255,255,0), 4 : (255,255,255), 5: (160,160,160), 6: (255,0,0), 7: (255,0,255), 8: (80,80,0), 9 : (80,0,0) }

In [ ]:
def display(points,types):
    m = np.max(np.abs(points))
    points = (points+m)/(2*m)
    v = 800
    graph = np.zeros((v,v,3),np.uint8)
    for i in range(len(points)):
        cv2.circle(graph,(int(v*points[i,0]),int(v*(1-points[i,1]))),2,colors[types[i]],cv2.FILLED)
        
    for j in range(10):
        cv2.rectangle(graph,(j*32,0),((j+1)*32,32),colors[j],cv2.FILLED)
        cv2.putText(graph,str(j),(j*32+8,32-8),0,0.9,(0,0,0))
    
    return graph

In [ ]:
mean, eigenvectors = cv2.PCACompute(feature_vectors, np.mean(feature_vectors, axis=0).reshape(1,-1))

In [ ]:
points = np.dot(feature_vectors-mean,eigenvectors.T)
points = points[:,:2]

In [ ]:
graph = display(points, values)
cv2.imwrite('latent-space.png',graph)
cv2.imshow('points',graph)
cv2.waitKey(0)
cv2.destroyAllWindows()